In [2]:
library(dbplyr)
library(DBI)
library(RSQLite)
library(dplyr)
library(SummarizedExperiment)
library(tidyr)

In [50]:
ExportSummarizedExperiment <- function(summary){
    ###  Check user input
    
    # read filtered summary file from user input
    #summary=read.csv("filter_summary.csv",header=T)
   

    
    #connecting to SQL database in R
    con <- DBI::dbConnect(RSQLite::SQLite(), dbname = "../data/mprabase_v4_6.db")

    SP_n=nrow(summary) ## number of samples

## loop over all samples in the summary file
    for(i in 1:SP_n) {

    PMID=summary$PMID[i]
    SPID=summary$sample_name[i]
    #### read info for metaData (PMID,GEO_number,SRP_number,labs,Organism,Cell_line_tissue,sample_name,description,DNA_RNA_reps,element_position,rep_type,number_of_elements,Reference)
    sqlStatement_coldata <- paste(" SELECT datasets.PMID, datasets.GEO_number, datasets.SRP_number, datasets.labs, sample.Library_strategy, sample.Organism, sample.Cell_line_tissue,sample.sample_name, sample.description,sample.DNA_RNA_reps, sample.element_position,sample.rep_type, designed_library.number_of_elements, datasets.Reference FROM datasets INNER JOIN designed_library ON datasets.datasets_id = designed_library.datasets_id INNER JOIN  sample ON designed_library.library_id=sample.library_id WHERE  datasets.PMID =","'",PMID,"'","AND  sample.sample_name=","'",SPID,"'",sep="")
   
    metaData =  dbGetQuery(con,sqlStatement_coldata)
    colData={}
    ## generate colData for each rep  
    for (j in 1:metaData$DNA_RNA_reps){
    colData=rbind(colData,data.frame(REP=j,
                                     condition=metaData$description,
                                     rep_type=metaData$rep_type,
                                     Organism=metaData$Organism,
                                     Cell_line_tissue=metaData$Cell_line_tissue))
     }
    ### read data for assay (ActivityScore)
    sqlStatement_assay <- paste("  SELECT library_sequence.library_element_name, library_sequence.element_coordinate, library_sequence.sequence, library_sequence.genome_build, element_rep_score.REP1, element_rep_score.REP2, element_rep_score.REP3 FROM datasets  INNER JOIN designed_library ON datasets.datasets_id = designed_library.datasets_id INNER JOIN sample ON designed_library.library_id = sample.library_id INNER JOIN  element_rep_score ON sample.sample_id=element_rep_score.sample_id  INNER JOIN library_sequence ON library_sequence.library_element_id = element_rep_score.library_element_id WHERE  datasets.PMID =","'",PMID,"'","AND  sample.sample_name=","'",SPID,"'",sep="")   
    assay_table<- dbGetQuery(con, sqlStatement_assay)



    ###need to add if it is genomic coordinates
    ## making Grange
    coord_all_table <- separate(data = assay_table, col = element_coordinate, into= c("seqnames","start","end"))
    gr = GRanges(seqnames = as.character(unlist(coord_all_table$seqnames)), 
    ranges = IRanges(as.numeric(unlist(coord_all_table$start)),
    end=as.numeric(unlist(coord_all_table$end)),
    names = unlist(coord_all_table$library_element_name)))
    mcols(gr) = subset(coord_all_table,select=c(genome_build,sequence))
 
    
    
    #### making SummarizedExperiment
    SE1=SummarizedExperiment(assays=list(ActivityScore=(cbind(coord_all_table$REP1,
                                                              coord_all_table$REP2,
                                                              coord_all_table$REP3))),
                             rowRanges=gr,
                             colData=colData)
    metadata(SE1)=metaData
  ## for muliple sample, return user a SummarizedExperiment list
   if(i <=1) {SE=SE1 
              SE=list(SE)
             } 
        else {SE = append(SE, SE1)}
    }
    return(SE)

}


In [51]:
# load summary file
summary <- read.table("../inst/summary.csv", header=T, sep=',')

In [52]:
user_summary_dataframe <- filter(summary, GEO_number=="GSE83894")

In [53]:
head(user_summary_dataframe)

,PMID,GEO_number,SRP_number,labs,sample_name,number_of_elements,Library_strategy,Organism,Cell_line_tissue,DNA_RNA_reps
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,27831498,GSE83894,,"Shendure,Ahituv",Mutant_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
2,27831498,GSE83894,,"Shendure,Ahituv",Wild-type_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3


In [82]:
# Actions Items: 11.10.22
# 1. assays(se[[1]])$ActivityScore - add sample names to column
# 2. update env to support reference builds - example: library(BSgenome.Hsapiens.UCSC.hg38)
# 3. update colData column names: Replicate-ID, Condition, Replicate-Type, Organism, Cell-Line-Tissue
# 4. Generate 50 "high quality" SE objects (ideally with strand + oligos with & without adaptors)
# 5. Condition 1: No genomic coordinates (synthetic oligos)
#    Replace rowRanges with a dataframe (rows- element names, column (col.names: sequence)- raw sequence)
# 6. Condition 2: mutagenesis (not original sequence)
#    Are the ranges reported, REF/ALT allele + position (ideally strand- defined by the target (e.g., gene or transcript)?
  
# Example: how to pull sequence information
# candidate_control_seqs <- BSgenome::getSeq(BSgenome.Hsapiens.UCSC.hg38,
#                                            seqnames(promoters),
#                                            start=start(promoters), end=end(promoters),
#                                            strand=strand(promoters))


se <- ExportSummarizedExperiment(user_summary_dataframe)

In [79]:
rowData(se[[1]]) <- as.data.frame(rep('TCGA',dim(rowData(se[[1]]))[1]))

In [ ]:
se

[[1]]
class: RangedSummarizedExperiment 
dim: 2440 3 
metadata(14): PMID GEO_number ... number_of_elements Reference
assays(1): ActivityScore
rownames(2440):
  A:HNF4A-ChMod_chr10:11917871-11917984_[chr10:11917842-11918013]
  A:HNF4A-ChMod_chr10:34165653-34165745_[chr10:34165613-34165784] ...
  R:HNF4A-NoMod_chr9:98508868-98508933_[chr9:98508815-98508986]
  R:HNF4A-NoMod_chrY:18213828-18213963_[chrY:18213810-18213981]
rowData names(2): genome_build sequence
colnames: NULL
colData names(5): REP condition rep_type Organism Cell_line_tissue

[[2]]
class: RangedSummarizedExperiment 
dim: 2440 3 
metadata(14): PMID GEO_number ... number_of_elements Reference
assays(1): ActivityScore
rownames(2440):
  A:HNF4A-ChMod_chr10:11917871-11917984_[chr10:11917842-11918013]
  A:HNF4A-ChMod_chr10:34165653-34165745_[chr10:34165613-34165784] ...
  R:HNF4A-NoMod_chr9:98508868-98508933_[chr9:98508815-98508986]
  R:HNF4A-NoMod_chrY:18213828-18213963_[chrY:18213810-18213981]
rowData names(2): genome_build se

In [19]:
se[[1]]

class: RangedSummarizedExperiment 
dim: 2440 3 
metadata(14): PMID GEO_number ... number_of_elements Reference
assays(1): ActivityScore
rownames(2440):
  A:HNF4A-ChMod_chr10:11917871-11917984_[chr10:11917842-11918013]
  A:HNF4A-ChMod_chr10:34165653-34165745_[chr10:34165613-34165784] ...
  R:HNF4A-NoMod_chr9:98508868-98508933_[chr9:98508815-98508986]
  R:HNF4A-NoMod_chrY:18213828-18213963_[chrY:18213810-18213981]
rowData names(2): genome_build sequence
colnames: NULL
colData names(5): REP condition rep_type Organism Cell_line_tissue

In [37]:
# Example: for condition 1 (no rowRanges slot) use the assays column and set name to SyntheticElement
assays(se[[1]])$'SyntheticElements' <- assays(se[[1]])$ActivityScore-1

In [24]:
colData(se[[1]]) 

DataFrame with 3 rows and 5 columns
        REP        condition    rep_type     Organism Cell_line_tissue
  <integer>      <character> <character>  <character>      <character>
1         1 Mutant integrase          NA Homo sapiens            HepG2
2         2 Mutant integrase          NA Homo sapiens            HepG2
3         3 Mutant integrase          NA Homo sapiens            HepG2

In [26]:
metadata(se[[1]])

PMID,GEO_number,SRP_number,labs,Library_strategy,Organism,Cell_line_tissue,sample_name,description,DNA_RNA_reps,element_position,rep_type,number_of_elements,Reference
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
27831498,GSE83894,,"Shendure,Ahituv",lentiMPRA,Homo sapiens,HepG2,Mutant_integrase_HepG2,Mutant integrase,3,5'/3',NA,2440,A systematic comparison reveals substantial differences in chromosomal versus episomal encoding of enhancer activity
